In [1]:
import pandas as pd
df = pd.read_csv('D:\music rec\spotify_millsongdata.csv')
df.head

<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\kalya\AppData\Local\Temp\ipykernel_348\89468610.py:2: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv('D:\music rec\spotify_millsongdata.csv')


<bound method NDFrame.head of              artist                   song  \
0              ABBA  Ahe's My Kind Of Girl   
1              ABBA       Andante, Andante   
2              ABBA         As Good As New   
3              ABBA                   Bang   
4              ABBA       Bang-A-Boomerang   
...             ...                    ...   
57645  Ziggy Marley          Good Old Days   
57646  Ziggy Marley          Hand To Mouth   
57647          Zwan           Come With Me   
57648          Zwan                 Desire   
57649          Zwan              Heartsong   

                                              link  \
0       /a/abba/ahes+my+kind+of+girl_20598417.html   
1            /a/abba/andante+andante_20002708.html   
2             /a/abba/as+good+as+new_20003033.html   
3                       /a/abba/bang_20598415.html   
4           /a/abba/bang+a+boomerang_20002668.html   
...                                            ...   
57645  /z/ziggy+marley/good+old+days_10

In [2]:
df.tail

<bound method NDFrame.tail of              artist                   song  \
0              ABBA  Ahe's My Kind Of Girl   
1              ABBA       Andante, Andante   
2              ABBA         As Good As New   
3              ABBA                   Bang   
4              ABBA       Bang-A-Boomerang   
...             ...                    ...   
57645  Ziggy Marley          Good Old Days   
57646  Ziggy Marley          Hand To Mouth   
57647          Zwan           Come With Me   
57648          Zwan                 Desire   
57649          Zwan              Heartsong   

                                              link  \
0       /a/abba/ahes+my+kind+of+girl_20598417.html   
1            /a/abba/andante+andante_20002708.html   
2             /a/abba/as+good+as+new_20003033.html   
3                       /a/abba/bang_20598415.html   
4           /a/abba/bang+a+boomerang_20002668.html   
...                                            ...   
57645  /z/ziggy+marley/good+old+days_10

In [3]:
df.shape

(57650, 4)

In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
df = df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [6]:
df.head(10)

,artist,song,text
0,Reo Speedwagon,Someone Tonight,You've got it. I want it. \r\nSurrender. Don'...
1,Queen,Now I'm Here,Here I stand (here I stand) \r\nLook around a...
2,Yonder Mountain String Band,How 'bout You,Don't tie me up and take my time \r\nWaiting ...
3,Rod Stewart,Broken Arrow,Who else is gonna bring you a broken arrow \r...
4,Faces,Pool Hall Richard,"Sweatin' hard, I didn't get a shot \r\nAll I ..."
5,Sam Smith,I've Told You Now,You know what I mean \r\nIt's like walking in...
6,Queen Latifah,How Long (Betcha' Got A Chick On The Side),"Betcha got a chick on the side, sure you got a..."
7,Rush,Ghost Of A Chance,Like a million little doorways \r\nAll the ch...
8,Unwritten Law,Sound Siren,Sail with me across the sea \r\nSo we could s...
9,Chicago,Long Lost Friend,The moment that we met \r\nThere was somethin...


In [7]:
df.shape

(5000, 3)

In [8]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')

def tokenization(txt):
    tokens = word_tokenize(txt)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

In [10]:
df['text'] = df['text'].apply(tokenization)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
df['text'] = df['text'].apply(lambda x: ' '.join(x))

In [13]:
tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [14]:
similarity[0]

array([1.        , 0.02516543, 0.06493792, ..., 0.05808935, 0.00811932,
       0.02429154])

In [15]:
df[df['song'] == 'Street']

,artist,song,text


In [16]:
def recommendation(song_df):
    try:
        idx = df[df['song'] == song_df].index[0]
    except IndexError:
        print(f"Song '{song_df}' not found in the dataset.")
        return []
    
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [17]:
recommendation('love')

Song 'love' not found in the dataset.


[]

In [18]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [19]:
model = pd.read_pickle("df.pkl")